Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [0]:
data_root = './data' # Change me to store data elsewhere

In [0]:
url = 'http://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(dest_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', dest_filename)
  else:
    raise Exception(
      'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [0]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(path=data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception( 
     'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [0]:
# Display few files randomly chosen
import random
from IPython.display import Image, display

def display_files(folder_list, num):
  folder = train_folders[random.randint(0, len(train_folders)-1)]
  file_list = os.listdir(folder)
  for i in range(num):
    fname = os.path.join(folder,file_list[random.randint(0, len(file_list))])
    assert os.path.isfile(fname)
    print("Display {}".format(fname))
    display(Image(filename=fname))

display_files(train_folders, 5)
display_files(test_folders, 5)    

Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the r
ad. 

A few images might not be readable, we'll just skip them.

In [0]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset

# Pickle is python specific serialization/deserialization utility.
# One can even represent pointer sharing albeit only within python programs
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        # 'b' is for binary mode - python on windows distinguishes
        # between text/binary files
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

In [0]:
index_to_letter = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'] 
fig, axs = plt.subplots(nrows=4, ncols=5, sharex=True, figsize=(10, 5))
for row in range(4):
  for col in range(5):
    axs[row][col].set_title(index_to_letter[(row*5+col)%10])

def get_dataset_info_n_sample(datasets, label):
  pickle_file = datasets[label]
  with open(pickle_file, 'rb') as f:
    letter_set = pickle.load(f)
    num_imgs = letter_set.shape[0]
    index = random.randint(0,num_imgs-1)
    return num_imgs, letter_set[index]

# Extract number of valid images we have for each Class 
train_letter_rows = []
test_letter_rows = []

# Display random image for each letter from each dataset
for label in range(10):
  num_imgs, img = get_dataset_info_n_sample(train_datasets, label)
  train_letter_rows.append(num_imgs)
  axs[label/5][label%5].imshow(img)
  num_imgs, img = get_dataset_info_n_sample(test_datasets, label)
  test_letter_rows.append(num_imgs)
  axs[label/5 + 2][label%5].imshow(img)  

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

In [0]:
# Data Balance Check accross class:
max_divergence_ratio_allowed = 1.5 # allow min vs max counts to not diverge beyond 40%

In [0]:
print("Train Letter Rows={}: Test Letter Rows={}", train_letter_rows, test_letter_rows)
train_num_imgs = np.array(train_letter_rows, dtype=np.float32)
test_num_imgs  = np.array(test_letter_rows, dtype=np.float32)
divergence_ratios = []
divergence_ratios.append(1 - np.min(train_num_imgs/np.max(train_num_imgs)))
divergence_ratios.append(np.max(train_num_imgs/np.min(train_num_imgs)) - 1)
divergence_ratios.append(1 - np.min(test_num_imgs/np.max(test_num_imgs)))
divergence_ratios.append(np.max(test_num_imgs/np.min(test_num_imgs)) - 1)
divergence_ratio_arr = np.array(divergence_ratios)
max_ratio = np.max(divergence_ratio_arr)
print("max divergence ratio={}: divergence ratios={}".format(max_ratio, divergence_ratio_arr))
if max_ratio > max_divergence_ratio_allowed:
  print("data too imbalanced: ratios of min-max {} > {}".format(max_ratio, max_divergence_ratio_allowed))

Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `train_size` as needed. The labels will be stored into 

 separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [0]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [0]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

---
# Data is good: i.e. images are valid & data is well balanced

In [0]:
# Check Well Balanced Data
def imbalance_ratio(labels):
  _, counts = np.unique(labels, return_counts=True)
  print("Counts for each label={}".format(counts))
  counts = counts.astype(np.float32)
  if (counts.shape[0] < num_classes):
    return max_divergence_ratio_allowed + 1.0
  counts_min =  np.max(counts/np.min(counts) - 1)
  counts_max = np.min(1 - counts/np.max(counts))
  return max(counts_min, counts_max)

train_imb = imbalance_ratio(train_labels)
valid_imb = imbalance_ratio(valid_labels)
test_imb = imbalance_ratio(test_labels)
print("Max Imbalance Ratio Allowed: {}".format(max_divergence_ratio_allowed))
print("Train Imbalance Ratio: {}".format(train_imb))
print("Validation Imbalance Ratio: {}".format(valid_imb))
print("Test Imbalance Ratio: {}".format(test_imb))
if train_imb > max_divergence_ratio_allowed or \
  valid_imb > max_divergence_ratio_allowed or \
  test_imb > max_divergence_ratio_allowed:
  print("Train Imb {}, Valid Imb {}, or Test Imb {} > {} max allowed".\
        format(train_imb, valid_imb, test_imb, max_divergence_ratio_allowed))

In [0]:
# Images Good: Display nrows*ncols random images
def disp_images(dataset, labels, nrows, ncols):
  num_images = nrows*ncols
  fig, axs = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, figsize=(10, 5))
  num_data = dataset.shape[0]
  for row in range(nrows):
    for col in range(ncols):
      index = random.randint(0,num_data-1)
      img = dataset[index]
      label = labels[index]
      axs[row][col].set_title(index_to_letter[label])
      axs[row][col].imshow(img)      

In [0]:
disp_images(train_dataset, train_labels, 2, 5)

In [0]:
disp_images(valid_dataset, valid_labels, 2, 5)      

In [0]:
disp_images(test_dataset, test_labels, 2, 5)     

Finally, let's save the data for later reuse:

In [0]:
pickle_file = os.path.join(data_root, 'notMNIST.pickle')

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [0]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [0]:
pickle_file = os.path.join(data_root, 'notMNIST.pickle')
all_data = pickle.load(open(pickle_file, 'rb'))
train_dataset = all_data['train_dataset']
train_labels = all_data['train_labels']
test_dataset = all_data['test_dataset']
test_labels = all_data['test_labels']

In [0]:
# Before Overlap Test we need to normalize data.
# Although element (pixel) wise normalization of each datapoint
# with mean 0 and std-dev 1/2 has been done, we have not
# normalized the entire vector of image (28*28) pixel.
# Overlap between Dataset A and B is identify dot product of each row of A with row of B.
# Eliminate High values of B (> threshold) where dot products are high
# High Threshold
x = train_dataset.reshape(-1,28*28)
train_mat = x/np.sqrt(np.sum(x*x,axis=1).reshape(-1,1))
x = valid_dataset.reshape(-1,28*28)
val_mat = x/np.sqrt(np.sum(x*x,axis=1).reshape(-1,1))
x  = test_dataset.reshape(-1,28*28)
test_mat = x/np.sqrt(np.sum(x*x,axis=1).reshape(-1,1))


In [0]:
# Train Dataset and Validation Dataset Overlap
x_tr_val = np.abs(train_mat.dot(val_mat.T))
# Validation Dataset and Test Dataset Overlap
x_val_test = np.abs(val_mat.dot(test_mat.T))
# Train Dataset and Test Dataset Overlap
x_test_tr = np.abs(test_mat.dot(train_mat.T))

In [0]:
# Filter out any image rows that exceed Similarity Threshold of other datasets
max_similarity_allowed = 0.95
tr_val_sim = ~np.any(x_tr_val > max_similarity_allowed, axis=1)
val_test_sim = ~np.any(x_val_test > max_similarity_allowed, axis=1)
test_tr_sim = ~np.any(x_test_tr > max_similarity_allowed, axis=1)

In [0]:
train_dataset = train_dataset[tr_val_sim]
valid_dataset = valid_dataset[val_test_sim]
test_dataset = test_dataset[test_tr_sim]

In [0]:
train_labels = train_labels[tr_val_sim]
valid_labels = valid_labels[val_test_sim]
test_labels = test_labels[test_tr_sim]

In [0]:
print("# rows - train_dataset {} train_labels {}".format(len(train_dataset), len(train_labels)))
print("# rows - valid_dataset {} valid_labels {}".format(len(valid_dataset), len(valid_labels)))
print("# rows - test_dataset {} test_labels {}".format(len(test_dataset), len(test_labels)))

In [0]:
# Data balanced after removing similarities?
train_imb = imbalance_ratio(train_labels)
valid_imb = imbalance_ratio(valid_labels)
test_imb = imbalance_ratio(test_labels)
print("Max Imbalance Ratio Allowed: {}".format(max_divergence_ratio_allowed))
print("Train Imbalance Ratio: {}".format(train_imb))
print("Validation Imbalance Ratio: {}".format(valid_imb))
print("Test Imbalance Ratio: {}".format(test_imb))
if train_imb > max_divergence_ratio_allowed or \
  valid_imb > max_divergence_ratio_allowed or \
  test_imb > max_divergence_ratio_allowed:
  print("Train Imb {}, Valid Imb {}, or Test Imb {} > {} max allowed".\
        format(train_imb, valid_imb, test_imb, max_divergence_ratio_allowed))

---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data
---

In [0]:
def train_n_test(num_samples):
  train_samples = np.random.randint(0,len(X_train),num_samples)  
  logreg = LogisticRegression(n_jobs=-1)
  logreg.fit(X_train[train_samples], train_labels[train_samples])
  val_acc = logreg.score(X_valid, valid_labels)
  test_acc = logreg.score(X_test, test_labels)
  print("Test accuracy: # Train Samples = {}. Validation Accuracy={}. Test Accuracy={}".\
        format(num_samples, val_acc, test_acc))

X_train = train_dataset.reshape(-1, 28*28)
X_test = test_dataset.reshape(-1, 28*28)
X_valid = valid_dataset.reshape(-1,28*28)

n_sample_list = [50, 100, 1000, 5000] # len(X_train)]
for num_samples in n_sample_list:
  train_n_test(num_samples)